In [8]:
import pulp


#################################################################
# Simple Example 
# T = range(1, 10)  # Time horizon: 1 to 336

# J = {
#     # Job ID: (Part ID, Due minutes)
#     4421001: ('MP0389', 8),
#     4421002: ('MP0389', 8)     
# }

# part_to_tasks = {
#     'MP0389': [1, 2]
# }

# compat2 = {
#     # Task : Machines
#     1:[1, 2],
#     2:[2]
# }

# numMachines = 2
# M = range(1, numMachines + 1)

# dur ={
#     ('MP0389', 1): 2,
#     ('MP0389', 2): 3
# }
#################################################################

#################################################################
# First Example 
# T = range(1, 25)  # Time horizon: 1 to 336

# J = {
#     # Job ID: (Part ID, Due minutes)
#     1: ('MP0389', 8),
#     2: ('MP0389', 8)     
# }

# part_to_tasks = {
#     'MP0389': [1, 2, 3]
# }

# compat2 = {
#     # Task : Machines
#     1:[1, 2, 3],
#     2:[1, 2, 3],
#     3:[1, 2, 3]
# }

# numMachines = 3
# M = range(1, numMachines + 1)

# dur ={
#     ('MP0389', 1): 2,
#     ('MP0389', 2): 3,
#     ('MP0389', 3): 4
# }
#################################################################

#################################################################
# Second Example 
T = range(1, 20)  # Time horizon: 1 to 336

J = {
    # Job ID: (Part ID, Due minutes)
    1: ('MP0189', 10),
    2: ('MP0289', 10),
    3: ('MP0389', 10)
}

part_to_tasks = {
    'MP0189': [1, 2, 3],
    'MP0289': [4, 5],
    'MP0389': [6, 7, 8],
}

compat2 = {
    # Task : Machines
    1:[1],
    2:[1],
    3:[1],
    4:[1, 2],
    5:[1, 2],
    6:[1, 3],
    7:[1],
    8:[2, 3]
}

numMachines = 3
M = range(1, numMachines + 1)

dur ={
    ('MP0189', 1): 2,
    ('MP0189', 2): 2,
    ('MP0189', 3): 2,
    
    ('MP0289', 4): 5,
    ('MP0289', 5): 2,
    
    ('MP0389', 6): 2,
    ('MP0389', 7): 2,
    ('MP0389', 8): 2
}
#################################################################

#y = pulp.LpVariable.dicts("y", ((j, k, M[m][0], t) for j in J for k in K[j] for m in M for t in T if (M[m][0] in compat[j][k-1] and t <= len(T)- d[j, k]+1)), 0, 1, pulp.LpBinary)
#x = pulp.LpVariable.dicts("x", ((j, k, M[m][0], t) for j in J for k in K[j] for m in M for t in T if M[m][0] in compat[j][k-1]), 0, 1, pulp.LpBinary)

# Variable for the start times
y = pulp.LpVariable.dicts("y", ((jobID, k, m, t) 
                                for jobID, (partID, dueTime) in J.items()
                                for k in part_to_tasks[partID]
                                for m in M
                                for t in T
                                if m in compat2[k] and t <= len(T) - dur[(partID, k)] + 1), 0, 1, pulp.LpBinary)

# Variable for the other times
x = pulp.LpVariable.dicts("x", ((jobID, k, m, t) 
                                for jobID, (partID, dueTime) in J.items()
                                for k in part_to_tasks[partID]
                                for m in M
                                for t in T
                                if m in compat2[k]), 0, 1, pulp.LpBinary)

prob = pulp.LpProblem("SchedulingProblem", pulp.LpMaximize)

#prob.setObjective( sum(f[j] for j in J) - sum( (t+d[j, K[j][-1]]) * y[j, K[j][-1], M[m][0], t]  for j in J for m in M for t in T if (j, K[j][-1], M[m][0], t) in y.keys()))
prob.setObjective(sum(d for (_, d) in J.values()) 
                  - sum( (t + dur[(partID, part_to_tasks[partID][-1])]) *
                        y[jobID, part_to_tasks[partID][-1], m, t] 
                            for jobID, (partID, dueTime) in J.items() 
                            for m in M 
                            for t in T 
                            if (jobID, part_to_tasks[partID][-1], m, t) in y.keys()
                        )
                  ) 
    
# -- Constraints -- 
# Each task in each batch starts once and only on one machine
Cons1=[]
for j, (partID, dueTime) in J.items():
    for k in part_to_tasks[partID]:
        Cons1.append(pulp.lpSum(y[j, k, m, t] for m in M for t in T if (j, k, m, t) in y.keys()) == 1)
        prob += Cons1[-1]
                
# A task only occupies a machine for the duration of processing time if the task is set to start on said machine
Cons2=[]
for j, (partID, dueTime) in J.items():
    for k in part_to_tasks[partID]:
        for m in M:
            for t in T:
                if (j, k, m, t) in y.keys():
                    # Ensuring x values correspond correctly with the y start times
                    for t_prime in range(t, t + dur[(partID, k)]):
                        if (j, k, m, t_prime) in x.keys():
                            Cons2.append(x[j, k, m, t_prime] >= y[j, k,m, t])
                            prob += Cons2[-1]
                            
# Total time the task occupies the machines equals the processing duration
Cons3=[]
for j, (partID, dueTime) in J.items():
    for k in part_to_tasks[partID]:
        for m in M:
            if (j, k, m, t) in x.keys(): # <- Should this t be looped over?
                Cons3.append(pulp.lpSum(x[j, k, m, t] for t in T) == dur[(partID, k)])
                prob+=Cons3[-1]
                
# Only one machine can be used at a time
Cons4=[]
for t in T:
    for m in M:
        Cons4.append(pulp.lpSum(x[j, k, m, t] 
                                for j, (partID, dueTime) in J.items() 
                                for k in part_to_tasks[partID]
                                if (j, k, m, t) in x.keys()) <= 1)
        prob+=Cons4[-1]

# Tasks must be done in the order specified by the batch
Cons5=[]
for j, (partID, dueTime) in J.items():
    for k in part_to_tasks[partID]:
        lastTaskForPart = part_to_tasks[partID][-1]
        if  k != lastTaskForPart:
            kp = part_to_tasks[partID].index(k)+1 # The index of the next task
            duration = dur[(partID, k)]
            nextTask = part_to_tasks[partID][kp]
                        
            Cons5.append(pulp.lpSum(t * y[j, k, m, t] for m in M for t in T if (j, k, m, t) in y.keys()) + duration <=
                         pulp.lpSum(t * y[j, nextTask, m, t] for m in M for t in T if (j, nextTask, m, t) in y.keys()))

            prob+=Cons5[-1]


# Solve the problem
print ("*************Model Defined **************")
prob.solve()

print("Status:", pulp.LpStatus[prob.status])

# Print the optimal value of the objective function
print("Optimal Value:", pulp.value(prob.objective))

# Print the results
# '''
for v in prob.variables():
    if v.varValue > 0.01: #and v.name.startswith('x') :
        print(f"{v.name} = {v.varValue}")
        
Data=[]

# Determine the time slots
for v in prob.variables():
    if v.varValue > 0.01 and v.name.startswith('y') :
        v=v.name.split(",")
                
        jobID = int(v[0].split("(")[1][:])
        task = int(v[1][1:])
        mach = int(v[2][1:])
        start = int(v[3][1:-1])
        partID = J[jobID][0]
        finish = start + dur[(partID, task)]        
        print(f"jobID: {jobID} m: {mach} t: {task} s: {start}")
        
        Data.append([mach ,f"J{jobID}-T{task}", start, finish])
        Time_Slot=[]
        for vp in prob.variables():
            if vp.varValue > 0.01 and vp.name.startswith('x') :
                vp=vp.name.split(",")
                jobID2 = int(vp[0].split("(")[1][:])
                task2 = int(vp[1][1:])
                mach2 = int(vp[2][1:])
                start2 = int(vp[3][1:-1])
                
                if ((mach, jobID, task) == (mach2, jobID2, task2)):
                    Time_Slot.append(start2)
        Data[-1].append(Time_Slot)
                
                
def Gant_Chart(Data):
    import pandas as pd
    mydata=pd.DataFrame(Data,columns=["machine","task" , "start" , "finish","Time Slot"])
    
    print(mydata)
    
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(10, 8))
    
    AllColors=[]
    import matplotlib.colors as mcolors
    for i in mcolors.TABLEAU_COLORS.keys():
        AllColors.append(i[4:])
    # for i in mcolors.CSS4_COLORS.keys():
    #     AllColors.append(i)
    # import random
    # random.shuffle(AllColors)
        
    machines = set(list(mydata["machine"]))
    tasks = list(set(list(mydata["task"])))
    # orign = min(list(mydata["start"])) # time origin
    # NM = len(machines) # Number of machines
    # NM=max(list (int(i[-1]) for i in machines))    
    NM = numMachines
    
    for i in range(1, numMachines + 1):
        # ymin = 0 # start y of gantt bar
        for index, row in mydata.iterrows():
            stdur = [] # list of tuples (start, duration)
            if row["machine"] == i:
                start = (row["start"])
                duration = (row["finish"] -  row["start"])
            
                stdur.append((start,duration))
                # print (stdur)
                # ax.annotate(row["task"], (start+duration/2,int(i[-1])))
                ax.annotate(row["task"], (start, i))
                ax.broken_barh(stdur, (i-0.25, 0.5), facecolors=(AllColors[tasks.index(row["task"])]))
           

    ax.set_ylim(0, NM+1)
    ax.set_yticks(range(1,NM+1))
    ax.set_xticks(T)
    # ax.set_yticklabels(machines)
    ax.grid(True)
    

    plt.show()
    return mydata

Data=Gant_Chart(Data)
Data=Data.sort_values("machine")
print(Data)
print("Status:", pulp.LpStatus[prob.status])
print("Optimal Value:", pulp.value(prob.objective))

        


Y equal
X equal
Sum lists equal are equal 
